In [23]:
from langchain.llms import HuggingFaceHub
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
import os
import qdrant_client


In [19]:
#creating the qdrant client
# qdrant_client = QdrantClient(
#     url=os.getenv('QDRANT_HOST'), 
#     api_key=os.getenv('QDRANT_API_KEY'),
#     )
client= qdrant_client.QdrantClient(
    url='https://26b3d6f5-2b4b-4439-b3ca-54ae3aee3147.us-east-1-0.aws.cloud.qdrant.io:6333', 
    api_key='1scJ0FLmw4GY2CyuFsfJh3FXnz4nwafwf8sZ1oXIdBAKaflWDGv9cw',
    )

In [20]:
#creating the collection
# collection_name=os.getenv('QDRANT_COLLECTION_NAME')
collection_name="test"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
    )

True

In [21]:
client.recreate_collection(
    collection_name="langchain",
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE),
)

True

In [25]:
embeddings = OpenAIEmbeddings()
vectorstore=Qdrant(
    client=client, collection_name="langchain", 
    embeddings=embeddings,
    )

In [27]:
from langchain.text_splitter import CharacterTextSplitter
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks
with open("test.txt", "r") as f:
    text = f.read()
text=get_text_chunks(text)
vectorstore.add_texts(text)

['66865ecb18b342c292185e32556183f7', 'ea870a286e5b462cb305b01e79abada3']

In [28]:
#plug vector store into retrieval chain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa=RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [30]:
query="What is the meaning of life?"
response=qa.run(query)
print(response)

 I don't know.
